## Data Cleaning

Let's load the some data for TSLA. Unfortunately, this data is not quite as *clean* as our NVDA data, so we'll need to do some data wrangling. The file we're looking to load is `TSLA_2015_2024.csv`.

In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("data/TSLA_2015_2024.csv")
df
df.Date = pd.to_datetime(df.Date, format="%d/%m/%Y")
df.set_index("Date", inplace=True)

Can you see what we mean by messy? How many issues can you spot?

- Dates out of order
- Duplicate rows
- Missing values

## Ordering and Duplicates

First let's start with sorting the index.

In [14]:
df.index.is_monotonic_increasing
# sort a dataframe 
df.sort_index(inplace=True)
df
df.index.is_monotonic_increasing

True

Now let's focus on duplicates:

In [19]:
df.duplicated().sum()
#getting rid of duplicates 
df.drop_duplicates(inplace=True)
df
df.duplicated().sum() #method chain
# inplace does not play well with data chain

np.int64(0)

#### Tip: Method Chaining

**Method chaining** is a popular feature of pandas. It allows us to *chain* together several operations in a single line of code. For example, we can set the index, sort the data frame and drop any duplicates all at once. Notice we don't use `inplace` but rather re-assign to the original `df` variable.

```python
df = df.set_index("Date").sort_index().drop_duplicates()
```

## Not a Number (NaN)

### Exercise: Some Null Chain

Let's look at the missing or `NaN` values next. Previously, we saw that `info()` gave us some insight into how many missing values we had, but we can also use `isnull()`.

Can you chain `isnull()` with `sum()` to get a single value stating the total number of missing values in the data frame?

In [ ]:
## YOUR CODE GOES HERE
df.isnull().sum().sum() # to get the total 
df

,Close,High,Low,Open,Volume
Date,,,,,
2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000.0
2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500.0
2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500.0
2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000.0
2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500.0
...,...,...,...,...,...
2024-12-23,430.600006,434.510010,415.410004,NaN,72698100.0
2024-12-24,462.279999,462.779999,435.140015,435.899994,59551800.0
2024-12-26,454.130005,465.329987,451.019989,465.160004,76366400.0


We can find out which rows have missing data using `isnull()`, `any()` along rows and some smart *masking*.

In [ ]:
condition = df.isnull().any(axis=1)
df[condition]

TypeError: 'DataFrame' object is not callable

Now that we've identified our missing values, the big question is how to handle them. There are many approaches to this that will vary depending on the data and the further analysis you plan to carry out.

In [ ]:
df.dropna(subset="Close") #to drop missing variables 
# missing values will still be present in the other places 
# when you want to fill your missing values 
    ## 1 Take mean of that column and use that to fill it in 
    df.Volume.fillna(df.Volume.mean())
    ##  linear method - interpolate by the two closest days near it 
    df.Close.interpolate(method="linear")
    ##forward fill - takes price from the day before 
    df.Close.ffill()

Date
2015-01-02     14.620667
2015-01-05     14.006000
2015-01-06     14.085333
2015-01-07     14.063333
2015-01-08     14.041333
                 ...    
2024-12-23    430.600006
2024-12-24    462.279999
2024-12-26    454.130005
2024-12-27    431.660004
2024-12-30    417.410004
Name: Close, Length: 2516, dtype: float64

### Exercise: Cleaning up

Notice how above we didn't actually update the `df` variable, so our DataFrame is still full of missing values. Fix all missing values applying the following rules:
- Fill missing Close by linear interpolation
- Fill missing Volume with the value from the day before
- Fill missing Open with the median Open
- Fill missing High with the Close or Open, whichever is higher
- Fill missing Low with a value 3% lower than the High


Your DataFrame `df` should have no missing values when done. Use `info()` to confirm.

**NOTE:** When changing values in a data frame, it is recommended to avoid using `inplace`, and instead re-assign the variable.

In [34]:
## YOUR CODE GOES HERE
df.Close = df.Close.interpolate(method="linear")

df.Volume = df.Volume.ffill()

df.Open = df.Open.fillna(df.Open.median())

df.High = df.High.fillna(df[["Close", "Open"]].max(axis=1)) #double square brackets because it is a list, axis is the columns 

df.Low = df.Low.fillna(df.High*0.97)

df.isnull().sum().sum()


np.int64(0)

#### Advanced: Data Types

You may have noticed that the **Volume** column in the 2021 data frame is a `float64` instead of the `int64` *dtype* we had in the 2020 data frame. Missing values (NaN) are represented as a special case of floating point number, so all the values in **Volume** were automatically *upcast* to floats.

Ideally our columns should be of the *dtype* that most accurately represents them. This will improve performance when working with large data frames. Now that we've resolved our missing numbers, we can *cast* our trading volumes as integers.

In [38]:
df.Volume = df.Volume.astype("int64")
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2516 entries, 2015-01-02 to 2024-12-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2516 non-null   float64
 1   High    2516 non-null   float64
 2   Low     2516 non-null   float64
 3   Open    2516 non-null   float64
 4   Volume  2516 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 117.9 KB


## Saving Data

Now that we've cleaned our data, let's save it, by writing it to a new .CSV file. We can use pandas' `to_csv()`.

In [43]:

df.to_csv("TSLA_clean.csv")
df

,Close,High,Low,Open,Volume
Date,,,,,
2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000
2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500
2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500
2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000
2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500
...,...,...,...,...,...
2024-12-23,430.600006,434.510010,415.410004,28.586000,72698100
2024-12-24,462.279999,462.779999,435.140015,435.899994,59551800
2024-12-26,454.130005,465.329987,451.019989,465.160004,76366400
